In [1]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.5 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=531a4c940a11e2fac539df6f3f5c4e4a2caccb1f5d8dd01e552c8e11db33a257
  Stored in directory: /root/.cache/pip/wheels/08/66/b4/921e351e63d88696932279d6163e125727c9da70ed8ca38419
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [2]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 3.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re
import csv
import swifter
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [19]:
!cp /content/gdrive/MyDrive/'Colab Notebooks'/'project'/data_ulasan_rev1.csv /content/
!cp /content/gdrive/MyDrive/'Colab Notebooks'/'project'/slangword.csv /content/

!ls

data_ulasan_rev1.csv  data_ulasan_rev.csv  gdrive  sample_data	slangword.csv


In [10]:
df = pd.read_csv('data_ulasan_rev1.csv', encoding = 'unicode_escape')

In [11]:
print('Dataset size:',df.shape)
print

Dataset size: (2000, 5)


<function print>

In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
df.head()

,userName,score,at,label_data,content
0,Witson Ferdinand,5,11/22/2022 6:48,positif,"Sangat bagus dan cepat,, semoga bisa membantu ..."
1,Putra Kurniawan,3,11/22/2022 6:47,positif,"pelayanannya cukup bagus, semoga semakin menin..."
2,Pandy syaputra Pandy,5,11/22/2022 6:42,positif,Sangat membantu dalam melakukan transferð¤ð...
3,Ai Nilam,5,11/22/2022 6:40,positif,sangat memudahkan kalau ingin bayar online
4,anisa haque,1,11/22/2022 6:40,negatif,"Tolong perbaiki, tadi lemot banget, jangan men..."


In [14]:
df = pd.DataFrame(df[['label_data', 'content']])
df

,label_data,content
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu ..."
1,positif,"pelayanannya cukup bagus, semoga semakin menin..."
2,positif,Sangat membantu dalam melakukan transferð¤ð...
3,positif,sangat memudahkan kalau ingin bayar online
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan men..."
...,...,...
1995,positif,Mantap.. Fiturnya lengkap.. Bagus sekali.. Suk...
1996,positif,Lebih cepat dan mudah dalam Proses transaksi d...
1997,negatif,Saya transfer dari malam 2 kali uang sudah dip...
1998,negatif,Saya sangat menyesal kemaren isi saldo di Ovo ...


In [15]:
#Cleansing text
def remove_punct(text):
    # menghilangkan url
    text = re.sub(r'https?:\/\/\S+','',text)
    # menghilangkan mention, link, hastag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    #menghilangkan karakter byte (b')
    text = re.sub(r'(b\'{1,2})',"", text)
    # menghilangkan yang bukan huruf
    text = re.sub('[^a-zA-Z]', ' ', text)
    # menghilangkan digit angka
    text = re.sub(r'\d+', '', text)
    #menghilangkan tanda baca
    text = text.translate(str.maketrans("","",string.punctuation))
    # menghilangkan whitespace berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

pd.set_option("display.max_colwidth", -1)
df['Cleantext'] = df['content'].apply(lambda x: remove_punct(x))
df.head()

,label_data,content,Cleantext
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu mempermudah transaksi",Sangat bagus dan cepat semoga bisa membantu mempermudah transaksi
1,positif,"pelayanannya cukup bagus, semoga semakin meningkat",pelayanannya cukup bagus semoga semakin meningkat
2,positif,Sangat membantu dalam melakukan transferð¤ð¤ð¤,Sangat membantu dalam melakukan transfer
3,positif,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan mengabaikan pesan dari pengguna sebagai saran",Tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran


In [16]:
#case folding
df['Case_Folding'] = df['Cleantext'].str.lower()
df.head()

,label_data,content,Cleantext,Case_Folding
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu mempermudah transaksi",Sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,sangat bagus dan cepat semoga bisa membantu mempermudah transaksi
1,positif,"pelayanannya cukup bagus, semoga semakin meningkat",pelayanannya cukup bagus semoga semakin meningkat,pelayanannya cukup bagus semoga semakin meningkat
2,positif,Sangat membantu dalam melakukan transferð¤ð¤ð¤,Sangat membantu dalam melakukan transfer,sangat membantu dalam melakukan transfer
3,positif,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan mengabaikan pesan dari pengguna sebagai saran",Tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran


In [17]:
#tokenization

def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tokenization'] = df['Case_Folding'].apply(lambda x: tokenization(x.lower()))
df.head()

,label_data,content,Cleantext,Case_Folding,Tokenization
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu mempermudah transaksi",Sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,"[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]"
1,positif,"pelayanannya cukup bagus, semoga semakin meningkat",pelayanannya cukup bagus semoga semakin meningkat,pelayanannya cukup bagus semoga semakin meningkat,"[pelayanannya, cukup, bagus, semoga, semakin, meningkat]"
2,positif,Sangat membantu dalam melakukan transferð¤ð¤ð¤,Sangat membantu dalam melakukan transfer,sangat membantu dalam melakukan transfer,"[sangat, membantu, dalam, melakukan, transfer]"
3,positif,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,"[sangat, memudahkan, kalau, ingin, bayar, online]"
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan mengabaikan pesan dari pengguna sebagai saran",Tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,"[tolong, perbaiki, tadi, lemot, banget, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]"


In [20]:
kamus_normalisasi = pd.read_csv("slangword.csv")

kata_normalisasi_dict = {}

for index, row in kamus_normalisasi.iterrows():
    if row[0] not in kata_normalisasi_dict:
        kata_normalisasi_dict[row[0]] = row[1] 
        
def normalisasi_kata(document):
    return [kata_normalisasi_dict[term] if term in kata_normalisasi_dict else term for term in document]

df['konversi_slang'] = df['Tokenization'].apply(normalisasi_kata)
df.head()

,label_data,content,Cleantext,Case_Folding,Tokenization,konversi_slang
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu mempermudah transaksi",Sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,"[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]","[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]"
1,positif,"pelayanannya cukup bagus, semoga semakin meningkat",pelayanannya cukup bagus semoga semakin meningkat,pelayanannya cukup bagus semoga semakin meningkat,"[pelayanannya, cukup, bagus, semoga, semakin, meningkat]","[pelayanannya, cukup, bagus, semoga, semakin, meningkat]"
2,positif,Sangat membantu dalam melakukan transferð¤ð¤ð¤,Sangat membantu dalam melakukan transfer,sangat membantu dalam melakukan transfer,"[sangat, membantu, dalam, melakukan, transfer]","[sangat, membantu, dalam, melakukan, transfer]"
3,positif,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,"[sangat, memudahkan, kalau, ingin, bayar, online]","[sangat, memudahkan, kalau, ingin, bayar, online]"
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan mengabaikan pesan dari pengguna sebagai saran",Tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,"[tolong, perbaiki, tadi, lemot, banget, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]","[tolong, perbaiki, tadi, lama, sekali, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]"


In [21]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
#stopword_removal
stopword = nltk.corpus.stopwords.words('indonesian')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

df['Stopword_Removal'] = df['konversi_slang'].apply(lambda x: remove_stopwords(x))
df.head()

,label_data,content,Cleantext,Case_Folding,Tokenization,konversi_slang,Stopword_Removal
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu mempermudah transaksi",Sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,"[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]","[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]","[bagus, cepat, semoga, membantu, mempermudah, transaksi]"
1,positif,"pelayanannya cukup bagus, semoga semakin meningkat",pelayanannya cukup bagus semoga semakin meningkat,pelayanannya cukup bagus semoga semakin meningkat,"[pelayanannya, cukup, bagus, semoga, semakin, meningkat]","[pelayanannya, cukup, bagus, semoga, semakin, meningkat]","[pelayanannya, bagus, semoga, meningkat]"
2,positif,Sangat membantu dalam melakukan transferð¤ð¤ð¤,Sangat membantu dalam melakukan transfer,sangat membantu dalam melakukan transfer,"[sangat, membantu, dalam, melakukan, transfer]","[sangat, membantu, dalam, melakukan, transfer]","[membantu, transfer]"
3,positif,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,"[sangat, memudahkan, kalau, ingin, bayar, online]","[sangat, memudahkan, kalau, ingin, bayar, online]","[memudahkan, bayar, online]"
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan mengabaikan pesan dari pengguna sebagai saran",Tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,"[tolong, perbaiki, tadi, lemot, banget, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]","[tolong, perbaiki, tadi, lama, sekali, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]","[tolong, perbaiki, mengabaikan, pesan, pengguna, saran]"


In [23]:
stopword_removal = df[['Stopword_Removal']]

def fit_stopwords(text):
    text = np.array(text)
    text = ' '.join(text)
    return text

df['Stopword_Removal'] = df['Stopword_Removal'].apply(lambda x: fit_stopwords(x))
df.head()

,label_data,content,Cleantext,Case_Folding,Tokenization,konversi_slang,Stopword_Removal
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu mempermudah transaksi",Sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,"[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]","[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]",bagus cepat semoga membantu mempermudah transaksi
1,positif,"pelayanannya cukup bagus, semoga semakin meningkat",pelayanannya cukup bagus semoga semakin meningkat,pelayanannya cukup bagus semoga semakin meningkat,"[pelayanannya, cukup, bagus, semoga, semakin, meningkat]","[pelayanannya, cukup, bagus, semoga, semakin, meningkat]",pelayanannya bagus semoga meningkat
2,positif,Sangat membantu dalam melakukan transferð¤ð¤ð¤,Sangat membantu dalam melakukan transfer,sangat membantu dalam melakukan transfer,"[sangat, membantu, dalam, melakukan, transfer]","[sangat, membantu, dalam, melakukan, transfer]",membantu transfer
3,positif,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,"[sangat, memudahkan, kalau, ingin, bayar, online]","[sangat, memudahkan, kalau, ingin, bayar, online]",memudahkan bayar online
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan mengabaikan pesan dari pengguna sebagai saran",Tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,"[tolong, perbaiki, tadi, lemot, banget, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]","[tolong, perbaiki, tadi, lama, sekali, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]",tolong perbaiki mengabaikan pesan pengguna saran


In [24]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    return stemmer.stem(text)

df['Stemming'] = df['Stopword_Removal'].apply(lambda x: stemming(x))
df.head()

,label_data,content,Cleantext,Case_Folding,Tokenization,konversi_slang,Stopword_Removal,Stemming
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu mempermudah transaksi",Sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,"[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]","[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]",bagus cepat semoga membantu mempermudah transaksi,bagus cepat moga bantu mudah transaksi
1,positif,"pelayanannya cukup bagus, semoga semakin meningkat",pelayanannya cukup bagus semoga semakin meningkat,pelayanannya cukup bagus semoga semakin meningkat,"[pelayanannya, cukup, bagus, semoga, semakin, meningkat]","[pelayanannya, cukup, bagus, semoga, semakin, meningkat]",pelayanannya bagus semoga meningkat,layan bagus moga tingkat
2,positif,Sangat membantu dalam melakukan transferð¤ð¤ð¤,Sangat membantu dalam melakukan transfer,sangat membantu dalam melakukan transfer,"[sangat, membantu, dalam, melakukan, transfer]","[sangat, membantu, dalam, melakukan, transfer]",membantu transfer,bantu transfer
3,positif,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,"[sangat, memudahkan, kalau, ingin, bayar, online]","[sangat, memudahkan, kalau, ingin, bayar, online]",memudahkan bayar online,mudah bayar online
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan mengabaikan pesan dari pengguna sebagai saran",Tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,"[tolong, perbaiki, tadi, lemot, banget, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]","[tolong, perbaiki, tadi, lama, sekali, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]",tolong perbaiki mengabaikan pesan pengguna saran,tolong baik abai pesan guna saran


In [26]:
df['text_clean'] = df['Stemming']
df.head()

,label_data,content,Cleantext,Case_Folding,Tokenization,konversi_slang,Stopword_Removal,Stemming,text_clean
0,positif,"Sangat bagus dan cepat,, semoga bisa membantu mempermudah transaksi",Sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,sangat bagus dan cepat semoga bisa membantu mempermudah transaksi,"[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]","[sangat, bagus, dan, cepat, semoga, bisa, membantu, mempermudah, transaksi]",bagus cepat semoga membantu mempermudah transaksi,bagus cepat moga bantu mudah transaksi,bagus cepat moga bantu mudah transaksi
1,positif,"pelayanannya cukup bagus, semoga semakin meningkat",pelayanannya cukup bagus semoga semakin meningkat,pelayanannya cukup bagus semoga semakin meningkat,"[pelayanannya, cukup, bagus, semoga, semakin, meningkat]","[pelayanannya, cukup, bagus, semoga, semakin, meningkat]",pelayanannya bagus semoga meningkat,layan bagus moga tingkat,layan bagus moga tingkat
2,positif,Sangat membantu dalam melakukan transferð¤ð¤ð¤,Sangat membantu dalam melakukan transfer,sangat membantu dalam melakukan transfer,"[sangat, membantu, dalam, melakukan, transfer]","[sangat, membantu, dalam, melakukan, transfer]",membantu transfer,bantu transfer,bantu transfer
3,positif,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,sangat memudahkan kalau ingin bayar online,"[sangat, memudahkan, kalau, ingin, bayar, online]","[sangat, memudahkan, kalau, ingin, bayar, online]",memudahkan bayar online,mudah bayar online,mudah bayar online
4,negatif,"Tolong perbaiki, tadi lemot banget, jangan mengabaikan pesan dari pengguna sebagai saran",Tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,tolong perbaiki tadi lemot banget jangan mengabaikan pesan dari pengguna sebagai saran,"[tolong, perbaiki, tadi, lemot, banget, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]","[tolong, perbaiki, tadi, lama, sekali, jangan, mengabaikan, pesan, dari, pengguna, sebagai, saran]",tolong perbaiki mengabaikan pesan pengguna saran,tolong baik abai pesan guna saran,tolong baik abai pesan guna saran


In [30]:
df.to_csv('text_preprocessing.csv', encoding='utf8', index=True)

In [28]:
df = pd.DataFrame(df[['text_clean', 'label_data']])
df

,text_clean,label_data
0,bagus cepat moga bantu mudah transaksi,positif
1,layan bagus moga tingkat,positif
2,bantu transfer,positif
3,mudah bayar online,positif
4,tolong baik abai pesan guna saran,negatif
...,...,...
1995,mantap fitur lengkap bagus sukses ovo,positif
1996,cepat mudah proses transaksi bayar,positif
1997,transfer malam kali uang potong proses gagal langsung kembali cepat,negatif
1998,sesal kemarin isi saldo ovo,negatif


In [31]:
df.to_csv('dataa_clean.csv', encoding='utf8', index=True)